In [1]:
import numpy as np , logging , os
import tensorflow as tf
from tensorflow import keras

from utils import tf_formh5 , auxua as aux , globo , xdv


''' GPU CONFIGURATION '''
tf_formh5.set_tf_loglevel(logging.ERROR)
tf_formh5.tf.debugging.set_log_device_placement(False) #Enabling device placement logging causes any Tensor allocations or operations to be printed.
tf_formh5.set_memory_growth()
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

cudatoolkit               10.1.243            h8cb64d8_11    conda-forge
cudnn                     7.6.5.32             hc0a50b0_1    conda-forge
neptune-tensorflow-keras  2.1.0              pyhd8ed1ab_0    conda-forge
numpy                     1.19.5           py38hc896f84_4  
numpy-base                1.19.5           py38h21a3de8_4  
tensorflow                2.2.0           gpu_py38hb782248_0    anaconda
tensorflow-base           2.2.0           gpu_py38h83e3d50_0    anaconda
tensorflow-estimator      2.6.0            py38h709712a_0    conda-forge
tensorflow-gpu            2.2.0                h0d30ee6_0    anaconda

Num GPUs Available:  4
4 Physical GPUs, 4 Logical GPUs


In [2]:
''' get layer's weights'''

layer = keras.layers.Dense(3)
layer.build((None, 4))  # Create the weights
layer.trainable = False  # Freeze the layer

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

weights: 2
trainable_weights: 0
non_trainable_weights: 2


In [3]:
train_rgb_config = {
    "ativa" : 'relu',
    "optima" : 'sgd',
    "batch_type" : 0,   # =0 all batch have frame_max or video length // =1 last batch has frame_max frames // =2 last batch has no repetead frames
    "frame_max" : '4000',
    "ckpt_start" : f"{0:0>8}",  #used in train_model: if 00000000 start from scratch, else start from ckpt with config stated
    "epochs" : 30
}

model , model_name = tf_formh5.init_modelrgb(train_rgb_config,globo.RGB_PATH)


FORM_MODEL


	 {'ativa': 'relu', 'optima': 'sgd', 'batch_type': 0, 'frame_max': '4000', 'ckpt_start': '00000000', 'epochs': 30} 

	OPTIMA <tensorflow.python.keras.optimizer_v2.gradient_descent.SGD object at 0x7f40c8570d00> 
	ATIVA relu

	WEIGHTS from  /.rgb/1677498953.2416248_relu_sgd_0_4000_xdviolence.h5


In [4]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 120, 160, 3 0         
_________________________________________________________________
conv3d (Conv3D)              (None, None, 118, 158, 4) 220       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, None, 59, 79, 4)   0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, None, 57, 77, 8)   1160      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, None, 28, 38, 8)   0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, None, 26, 36, 16)  9232      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, None, 13, 18, 16)  0     

In [9]:
# freeze model except the classifier head
for layer in model.layers[:-2]:layer.trainable = False

# check
for layer in model.layers[:]:print(layer.name ,layer.trainable)

input_1 False
conv3d False
max_pooling3d False
conv3d_1 False
max_pooling3d_1 False
conv3d_2 False
max_pooling3d_2 False
lambda False
lstm False
global_max_pooling1d False
dense_1 True
dense_2 True
